In [2]:
import itertools
import math
import os
import sys

import numpy as np
import pandas as pd
from numba import njit
from plotly.subplots import make_subplots
from vectorbt import FigureWidget
from vectorbt.generic import nb as generic_nb
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import vectorbt as vbt
from lib.utils import file_to_data_frame, LR, ExtendedPortfolio, plot_series_vs_scatters, dropnaninf, get_best_index, \
    where_true_set_series, shift_np

In [ ]:
file = "/Users/pilo/development/itba/pf/Binance_Minute_OHLC_CSVs/3000/Binance_ADAUSDT_minute_3000.csv"
_, ohlcv = file_to_data_frame(file)
ohlcv. head()

In [24]:
fee = 0.001
close = ohlcv["Close"]
volume = ohlcv["Volume"]
lr_ind = LR.run(close)
print(lr_ind.lr.shape)

NameError: name 'ohlcv' is not defined

In [3]:
@njit
def signal_calculations(lr, lr_ma, lr_mstd, vol, vol_mstd, lr_thld, vol_thld):
    lr_thld_std = lr_thld * lr_mstd
    ups = lr_ma - lr_thld_std # lr_thld < 0
    downs = lr_ma + lr_thld_std
    lr_entries = np.where(lr < downs, True, False)
    lr_exits = np.where(lr > ups, True, False)
    vol_thld_std = vol_thld * vol_mstd
    vol_entries = np.where(vol > vol_thld_std, True, False)
    return ups, downs, lr_entries, lr_exits, vol_thld_std, vol_entries

@njit
def ma_mstd(shifted_lr, shifted_volume, lag):
    lr_ma = generic_nb.rolling_mean_nb(shifted_lr, lag)
    lr_mstd = generic_nb.rolling_std_nb(shifted_lr, lag)
    vol_mstd = generic_nb.rolling_std_nb(shifted_volume, lag)
    return lr_ma, lr_mstd, vol_mstd

@njit
def signals_nb(lr, shifted_lr, vol, shifted_vol, lr_thld, vol_thld, lag):
    lr_ma, lr_mstd, vol_mstd = ma_mstd(shifted_lr, shifted_vol, lag)
    _, _, lr_entries, lr_exits, _, vol_entries = signal_calculations(lr, lr_ma, lr_mstd, vol, vol_mstd, lr_thld, vol_thld)
    final_entries = lr_entries & vol_entries
    return final_entries, lr_exits

In [ ]:
ENTRY_SIGNALS = vbt.IndicatorFactory(
    input_names=['lr', 'shifted_lr', 'vol', 'shifted_vol'],
    param_names=['lr_thld', 'vol_thld', 'lag'],
    output_names=['entries','exits']
).from_apply_func(signals_nb, use_ray=True)
lr_thld = -np.linspace(0,5, 30, endpoint=True)
vol_thld = np.linspace(0,10, 30, endpoint=True)
lag = list(range(6,100, 2))
signals = ENTRY_SIGNALS.run(lr=lr_ind.lr, shifted_lr=shift_np(lr_ind.lr.to_numpy(), 1),
                            vol=volume, shifted_vol=shift_np(volume.to_numpy(), 1),
                            lr_thld=lr_thld, vol_thld=vol_thld, lag=lag,
                            param_product=True, short_name="signals")
signals.entries.head()

In [5]:
def signals_to_ones(entries, exits, columns=None):
    if columns is None:
        columns = entries.columns
    entries = np.where(entries == True, 1, np.nan)
    return pd.DataFrame(np.where(exits == True, -1, entries), columns=columns)

In [ ]:
all_signals = signals_to_ones(signals.entries, signals.exits)

In [ ]:
vol_plots = []

In [12]:
@njit
def k_mean(col, arr, *args):
    indexes = np.where(np.isfinite(arr))[0]
    close = args[0]
    n = 1
    adder = 0
    counter = 0
    while n < len(indexes):
        i = indexes[n]
        prev = indexes[n-1]
        if arr[prev] == 1 and arr[i] == -1:
            adder += math.log(close[i]/close[prev])
            counter += 1
        n +=1
    return adder / counter if counter != 0 else 0
entry_exit_lr_avg = all_signals.vbt.reduce(k_mean, close.to_numpy())
entry_exit_lr_avg.vbt.volume(title="sum(ln(P_exit/P_entry),...) / #Trades").show() # .write_html(f"{module_path}/ADA-lr_mean.html")
entry_exit_lr_avg[entry_exit_lr_avg > 0].vbt.volume(title="(sum(ln(P_exit/P_entry),...) / #Trades) > 0").show() # .write_html(f"{module_path}/ADA-lr_mean>0.html")

NameError: name 'all_signals' is not defined

In [ ]:
vol = entry_exit_lr_avg[entry_exit_lr_avg > 0].vbt.volume()


In [28]:
@njit
def positive_return_prob(col, arr, *args):
    indexes = np.where(np.isfinite(arr))[0]
    close = args[0]
    fee = args[1]
    n = 1
    adder = 0
    counter = 0
    while n < len(indexes):
        i = indexes[n]
        prev = indexes[n-1]
        #if entry then exit
        if arr[prev] == 1 and arr[i] == -1:
            # if positive return
            if (close[i]/close[prev]) * (1 - fee)**2 > 1:
                adder += 1
            counter += 1
        n +=1
    return adder / counter if counter != 0 else 0
entry_exit_prob_avg = all_signals.vbt.reduce(positive_return_prob, close.to_numpy(), fee)
entry_exit_prob_avg.vbt.volume().show()

NameError: name 'all_signals' is not defined

In [ ]:
vol_plots.append({
    "data":entry_exit_prob_avg,
    "title":"(#Positive_trades / #Trades)"
})
vol_plots.append({
    "data":entry_exit_prob_avg[entry_exit_prob_avg<1],
    "title":"(#Positive_trades / #Trades) < 1"
})
#entry_exit_prob_avg.vbt.volume(title=" #Positive_trades / #Trades").show() # .write_html(f"{module_path}/ADA-trade_probability.html")
#entry_exit_prob_avg[entry_exit_prob_avg<1].vbt.volume().show() # .write_html(f"{module_path}/ADA-trade_probability<1.html")

In [ ]:
specs = [[{'type': 'scene'}]] * len(vol_plots)
subplot_titles = list(map(lambda plot: [plot["title"]], vol_plots))
fig = vbt.make_subplots(rows=len(vol_plots), cols=1, specs=specs, vertical_spacing=0.05, subplot_titles=subplot_titles)
for i in range(len(vol_plots)):
    vol_plots[i]["data"].vbt.volume(add_trace_kwargs=dict(row=i+1, col=1), fig=fig)
fig.update_layout(width=750, height=650)

In [ ]:
fig.show()

In [15]:
@njit
def trades_count(col, arr):
    indexes = np.where(np.isfinite(arr))[0]
    n = 1
    counter = 0
    while n < len(indexes):
        i = indexes[n]
        prev = indexes[n-1]
        if arr[prev] == 1 and arr[i] == -1:
            counter += 1
        n +=1
    return counter
entry_exit_trades_count = all_signals.vbt.reduce(trades_count)
entry_exit_trades_count.vbt.volume().volume(title=" #Trades").show() # .write_html(f"{module_path}/ADA-trade_count.html")

NameError: name 'all_signals' is not defined

In [ ]:
best_indexes = entry_exit_prob_avg.nlargest(20, keep="all")

In [ ]:
# graficamos con el mejor de los resultados de la optimización
best_idx = best_indexes[-1]
best_lr_thld, best_vol_thld, best_lag = best_idx

In [ ]:
# recalculamos algunas cosas para graficar y entender lo que está pasando
ups, downs, lr_entries, lr_exits, vol_thld_std, vol_entries = signal_calculations(
    lr_ind.lr.to_numpy(),
    ma_mstd.lr_ma[best_lag].to_numpy(),
    ma_mstd.lr_mstd[best_lag].to_numpy(),
    volume.to_numpy(),
    ma_mstd.vol_mstd[best_lag].to_numpy(),
    best_lr_thld,
    best_vol_thld)

In [ ]:
def plot_series_vs_scatters(series_list: list, booleans_list):
    index = None
    series = series_list.pop(0)
    fig = series.vbt.plot()
    while len(series_list):
        series = series_list.pop(0)
        if not isinstance(series, pd.Series):
            series = pd.Series(series, index=index, copy=True)
        elif index is None:
            index = series.index
        series.vbt.plot(fig=fig)
    i = 1
    for scatter in booleans_list:
        if not isinstance(scatter, pd.Series):
            scatter = pd.Series(scatter, index=index, copy=True)
        elif index is None:
            index = series.index
        scatter = where_true_set_series(series, scatter)
        scatter.name = i
        i += 1
        fig = scatter.vbt.scatterplot(fig=fig)
    return fig

entries = signals.entries[best_idx]
plot_series_vs_scatters([lr_ind.lr, downs], []).show()

In [ ]:
plots = [
    plot_series_vs_scatters([lr_ind.lr, downs.T], []) #, downs, ups], [lr_entries, lr_exits]),
    #plot_series_vs_scatters([volume, vol_thld_std], [vol_entries]),
    #(entries.where(entries == True, np.nan)).vbt.scatterplot()
]

In [ ]:
def add_all_subplots(fig, row, col, list):
    for a in list:
        fig.add_trace(a, row=row, col=col)

fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    vertical_spacing=0.02)
for i in range(len(plots)):
    add_all_subplots(fig, i+1, 1, plots[i].data)

fig.update_layout(height=700, legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

In [ ]:
@njit
def double_multiplier_nb(values, x, y):
    return values*x, values*y

DOUBLE_MULTIPLIER = vbt.IndicatorFactory(
    input_names=['values'],
    param_names=['x', 'y'],
    output_names=['x_mu','y_mu']
).from_apply_func(double_multiplier_nb)
x = np.linspace(0,2,5, endpoint=True)
y = -np.linspace(0,2,5, endpoint=True)
tp_sl = DOUBLE_MULTIPLIER.run(mstd_ind.mstd, x, y, param_product=True, short_name="tp_sl")

In [ ]:
tp_exits = lr_ind.lr_above(tp_sl.x_mu)
sl_exits = lr_ind.lr_below(tp_sl.y_mu)
final_exits = tp_exits.vbt | sl_exits.vbt
final_exits.columns = final_exits.columns.rename("lag", level=-1)
final_exits.head()

In [29]:
lag = 2
p = math.e ** np.array([[5],[6],[6],[3],[2],[4],[7],[8],[6],[5],[4]])
v = np.array([[5.],[6.],[6.],[3.],[2.],[4.],[7.],[8.],[6.],[5.],[4.]])
e_ln = np.log(p)
e_lr = np.array([[np.nan],[1],[0],[-3],[-1],[2],[3],[1],[-2],[-1],[-1]])
e_shifted_lr = np.array([[np.nan],[np.nan],[1],[0],[-3],[-1],[2],[3],[1],[-2], [-1]])
e_lr_ma = np.array([[np.nan], [np.nan],[np.nan],[ 1/2], [-3/2], [-2],[ 1/2],[ 5/2], [2], [-1/2], [-3/2]])
e_lr_mstd = np.array([[np.nan], [np.nan],[np.nan], [0.5], [1.5], [1. ], [1.5], [0.5], [1. ], [1.5], [0.5]])
lr = LR.run(p).lr.to_numpy()
shifted_lr = shift_np(lr, 1)
shifted_vol = shift_np(v, 1)
e_vol_mstd = np.array([[np.nan], [np.nan], [0.5], [0. ], [1.5], [0.5], [1. ], [1.5], [0.5], [1. ], [0.5]])
lr_ma, lr_mstd, vol_mstd = ma_mstd(shifted_lr, shifted_vol, 2)
assert (np.isclose(lr, e_lr, equal_nan=True).all())
assert (np.isclose(shifted_lr, e_shifted_lr, equal_nan=True).all())
assert (np.isclose(lr_ma, e_lr_ma, equal_nan=True).all())
assert (np.isclose(lr_mstd, e_lr_mstd, equal_nan=True).all())
assert (np.isclose(vol_mstd, e_vol_mstd, equal_nan=True).all())

thld = -2
e_lr_entries = e_lr < e_lr_ma + thld * e_lr_mstd
e_lr_exits = e_lr > e_lr_ma - thld * e_lr_mstd
e_vol_entries = v > -thld * vol_mstd
e_final_entries = e_lr_entries & e_vol_entries
_, _, lr_entries, lr_exits, _, vol_entries = signal_calculations(lr, lr_ma, lr_mstd, v, vol_mstd, thld, -thld)
final_entries,_ = signals_nb(lr, shifted_lr, v, shifted_vol, thld, thld, lag)
assert (np.array_equal(lr_entries, e_lr_entries))
assert (np.array_equal(lr_exits, e_lr_exits))
assert (np.array_equal(vol_entries, e_vol_entries))
assert (np.array_equal(final_entries, e_final_entries))

e_all_signals_in_ones = np.array([[np.nan],[np.nan],[np.nan],[1],[np.nan],[-1],[np.nan],[1],[1],[np.nan],[np.nan]])
all_signals_in_ones = signals_to_ones(final_entries, lr_exits, columns=[(lag, thld)])
assert (np.array_equal(all_signals_in_ones, e_all_signals_in_ones, equal_nan=True))

e_trade_count = 1
e_positive_return = 1
e_avg_lr = math.log(math.e**4/math.e**3)
avg_lr = k_mean("", all_signals_in_ones.to_numpy().flatten(), p.flatten())
positive_return = positive_return_prob("", all_signals_in_ones.to_numpy().flatten(), p.flatten(), fee)
trade_count = trades_count("", all_signals_in_ones.to_numpy().flatten())
assert (avg_lr == e_avg_lr)
assert (positive_return == e_positive_return)
assert (trade_count == e_trade_count)

[3 5 7 8]
[ 148.4131591   403.42879349  403.42879349   20.08553692    7.3890561
   54.59815003 1096.63315843 2980.95798704  403.42879349  148.4131591
   54.59815003]
trade
